In [1]:
from dataclasses import dataclass
import itertools
import logging
import random
import math
import numpy as np
import json
import pickle
import time
import torch
import sys
import pdb
import os
os.chdir("/Users/guotianyu/GitHub/birth")

from omegaconf import OmegaConf
from pathlib import Path
from torch import nn, Tensor
from torch.nn import functional as F
from typing import List, Optional, Tuple

from data import *
from ihead_full_model import ModelArgs, Transformer
%load_ext autoreload
%autoreload 2

# Preprocess Meta

In [2]:
ds = MetaProcess(DataArgs())
ds.bos_num

1

In [4]:
import pickle
import yaml

meta_processed = ds.process()
config_path = "data/bos1_d0"
os.makedirs(config_path, exist_ok=True)
with open(os.path.join(config_path, 'meta.pickle'), 'wb') as handle:
    pickle.dump(meta_processed, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(os.path.join(config_path, 'config.yaml'), 'w') as f:
    yaml.dump(meta_processed, f, default_flow_style=False)



# load meta and test corresponding dgp

In [2]:
import pickle
import numpy as np

with open("/Users/guotianyu/GitHub/birth/data/meta_bos1_d0.pickle", "rb") as f:
    meta_info = pickle.load(f)

# data_cfg = OmegaConf.structured(meta_info)


ds = dormant_copy_2(DataArgs(), meta_info)




In [3]:
x = ds.gen_batch(np.random.default_rng(0), 64)

In [5]:
x[0]

array([65, 55, 59,  1, 59, 57, 58, 57, 53, 57, 58, 57,  0, 33, 15, 39, 15,
       53, 15, 20, 27,  1, 27, 30, 27, 26, 19, 59, 58, 59, 56, 47, 52,  1,
       52, 43, 52, 42, 53, 42,  1, 42,  1, 42,  1, 42, 53, 42, 39, 42, 50,
       50, 39, 50,  1, 50,  1, 50, 50, 39, 50, 43, 50,  1, 50, 42, 43, 42,
       57,  6,  0, 26, 53, 26, 53, 26, 19, 39, 19, 56, 43, 56, 43, 56, 53,
       56, 52, 47, 58, 47, 57, 58, 57,  0, 35, 53, 35, 39, 35, 47, 57, 43,
       57, 57, 58, 57, 43, 57, 58, 57, 53, 57,  1, 57,  0, 32, 46, 57,  1,
       57,  1, 57, 12,  1, 12,  0, 13, 54, 43, 54, 43, 54, 50, 53, 50, 39,
       50, 53, 50, 63,  1, 63,  6,  1,  6,  1,  6,  0, 32, 46, 53, 46, 43,
       46, 43, 46, 47, 58, 47, 58, 47, 52, 42,  5, 58,  5, 42,  1, 42, 53,
       42,  1, 42,  1, 42, 53, 42, 43, 42, 57, 53, 57,  1, 57, 47, 57,  1,
       57, 58, 57, 43, 57, 43, 57,  1, 57, 43, 57, 54, 59, 56, 39, 56, 47,
       45, 46, 43, 46,  1, 46, 47, 45, 43, 45, 52,  1, 52, 57, 42, 56, 53,
       56,  1, 56,  1, 56